In [1]:
import pandas as pd
import numpy as np
import pyranges
import os
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import glob
import random

In [2]:
Variable_K27 = pd.read_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/Variable_K27.tsv', sep = "\t")


In [3]:
# Variable_K27 = Variable_K27.iloc[0:40,]

In [4]:
LoF = pd.read_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/Loss_of_function.tsv', sep = "\t")
LoF.shape

(40, 41)

In [5]:
GoF = pd.read_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/Gain_of_function.bed', sep = "\t")
GoF.shape

(28, 41)

In [6]:
Allpeaks = pd.read_csv("/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/Values_only_ATAC.csv", sep = "\t")


In [7]:
exceptions = pd.concat([LoF, GoF, Variable_K27])[['Chr', 'Start','End']]

In [8]:
exceptions["ID"] = exceptions["Chr"].str.cat(exceptions[["Start", "End"]].astype(str), sep="-")

In [9]:
subset = Allpeaks[~Allpeaks["ID"].isin(exceptions['ID'])].sample(36)



In [10]:
subset['ID']

22273     chr5-151051441-151052351
20778     chr4-155442075-155442896
26160     chr7-140267363-140268295
21017           chr5-219469-220573
3518       chr10-32332029-32332987
13502      chr19-43624630-43625538
13955         chr2-8684591-8686215
27600     chr8-143568692-143569903
7600     chr12-126874112-126875093
14529       chr2-61905269-61906165
20631     chr4-143773736-143774675
14658       chr2-70984483-70985420
15322     chr2-162319426-162319639
9871       chr15-88630085-88630977
27901       chr9-33233738-33234649
700         chr1-43524543-43525477
19531     chr3-195177824-195178748
6504       chr12-30778921-30779783
10203        chr16-4614419-4615341
28008       chr9-37485459-37486339
14599       chr2-68232181-68233005
27739         chr9-4923907-4924795
11646      chr17-44279657-44280428
18949     chr3-143031573-143032528
28575     chr9-122264394-122265290
11190        chr17-7841620-7842614
21350       chr5-62403390-62404322
29128       chrX-40647143-40648038
27545     chr8-14045

In [11]:
#subset.to_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/Other_peaks_values_only.csv', sep = "\t", index=False)


In [12]:
# remove chr5-170248326-170249251??

In [13]:
LoF["ID"]=LoF["Chr"].str.cat(LoF[["Start", "End"]].astype(str), sep="-")
GoF["ID"]=GoF["Chr"].str.cat(GoF[["Start", "End"]].astype(str), sep="-")
Variable_K27["ID"]=Variable_K27["Chr"].str.cat(Variable_K27[["Start", "End"]].astype(str), sep="-")

In [14]:
other_ATAC = pd.read_csv("/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/Other_peaks_values_only.csv", sep = "\t")
LoF_ATAC = Allpeaks[Allpeaks["ID"].isin(LoF['ID'])]
GoF_ATAC = Allpeaks[Allpeaks["ID"].isin(GoF['ID'])]
Variable_K27_ATAC = Allpeaks[Allpeaks["ID"].isin(Variable_K27['ID'])]


In [15]:
all_peaks_K27 = pd.read_csv("/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/Values_only_H3K27ac_window500-600bp_ATAC_ID.csv", sep = "\t")
other_K27 = all_peaks_K27[all_peaks_K27["ID"].isin(other_ATAC['ID'])]
LoF_K27 = all_peaks_K27[all_peaks_K27["ID"].isin(LoF['ID'])]
GoF_K27 = all_peaks_K27[all_peaks_K27["ID"].isin(GoF['ID'])]
Variable_K27_K27 = all_peaks_K27[all_peaks_K27["ID"].isin(Variable_K27['ID'])]

In [16]:
Donors = LoF_K27.columns[1:]
Donors2 = LoF_ATAC.columns[1:]

In [17]:
a = LoF_K27[Donors].to_numpy()
b = LoF_ATAC[Donors2].to_numpy()
LoF = pd.DataFrame(np.divide(a, b, out = np.ones_like(a), where=b!=0))
LoF.columns = Donors2
LoF.insert(0, 'ID', LoF_K27['ID'].values)

In [18]:
a = GoF_K27[Donors].to_numpy()
b = GoF_ATAC[Donors2].to_numpy()
GoF = pd.DataFrame(np.divide(a, b, out = np.ones_like(a), where=b!=0))
GoF.columns = Donors2
GoF.insert(0, 'ID', GoF_K27['ID'].values)

In [19]:
a = other_K27[Donors].to_numpy()
b = other_ATAC[Donors2].to_numpy()
others = pd.DataFrame(np.divide(a, b, out = np.ones_like(a), where=b!=0))
others.columns = Donors2
others.insert(0, 'ID', other_K27['ID'].values)

In [20]:
LoF.insert(0,'group',"LoF")
GoF.insert(0,'group',"GoF")
others.insert(0,'group',"other")
forSOM = pd.concat([LoF, GoF, others])
forSOM.to_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/forSOM_3groups_K27divATAC.csv', sep = "\t", index=False)


In [21]:
forSOM = pd.concat([LoF, GoF, Variable_K27.iloc[0:40,], others])
forSOM.to_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/forSOM_3groups_K27divATAC.csv', sep = "\t", index=False)


In [22]:
a = other_K27[Donors].to_numpy() + 1
b = other_ATAC[Donors2].to_numpy() + 1

others2 = pd.DataFrame(np.multiply(a, b))
others2.columns = Donors2
others2.insert(0, 'ID', other_K27['ID'].values)

In [23]:
a = LoF_K27[Donors].to_numpy() +1
b = LoF_ATAC[Donors2].to_numpy() +1
LoF2 = pd.DataFrame(np.multiply(a, b))
LoF2.columns = Donors2
LoF2.insert(0, 'ID', LoF_K27['ID'].values)

In [24]:
a = GoF_K27[Donors].to_numpy() +1
b = GoF_ATAC[Donors2].to_numpy() +1
GoF2 = pd.DataFrame(np.multiply(a, b))
GoF2.columns = Donors2
GoF2.insert(0, 'ID', GoF_K27['ID'].values)

In [25]:
LoF2.insert(0,'group',"LoF")
GoF2.insert(0,'group',"GoF")
others2.insert(0,'group',"other")
forSOM2 = pd.concat([LoF2, GoF2, others2])
forSOM2.to_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/forSOM_3groups_K27byATAC.csv', sep = "\t", index=False)


In [26]:
forSOM2['group'].value_counts()

LoF      40
other    36
GoF      28
Name: group, dtype: int64

In [27]:
other_all = pd.merge(other_K27, other_ATAC, on="ID")
other_all.insert(0,'group',"other")
LoF_all = pd.merge(LoF_K27, LoF_ATAC, on="ID")
LoF_all.insert(0,'group',"LoF")
GoF_all = pd.merge(GoF_K27, GoF_ATAC, on="ID")
GoF_all.insert(0,'group',"GoF")
for_keras = pd.concat([other_all, LoF_all, GoF_all])


In [28]:
for_keras.to_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/forkeras_3groups_K27_ATAC.csv', sep = "\t", index=False)


In [29]:
other_all = pd.merge(other_K27, other_ATAC, on="ID")
other_all.insert(0,'group',"other")
LoF_all = pd.merge(LoF_K27, LoF_ATAC, on="ID")
LoF_all.insert(0,'group',"LoF")
GoF_all = pd.merge(GoF_K27, GoF_ATAC, on="ID")
GoF_all.insert(0,'group',"GoF")
LoK27_all = pd.merge(Variable_K27_K27, Variable_K27_ATAC, on="ID")
LoK27_all.insert(0,'group',"LoK27")

for_keras2 = pd.concat([other_all, LoF_all, GoF_all, LoK27_all.iloc[0:40,]])
for_keras2.to_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/forkeras_4groups_K27_ATAC.csv', sep = "\t", index=False)


In [ ]:
for_keras3 = pd.concat([LoF_all, LoK27_all.iloc[0:40]])
for_keras3.to_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/forkeras_2groups_K27_LoK27.csv', sep = "\t", index=False)


In [ ]:

LoF_all = pd.merge(LoF_K27, LoF_ATAC, on="ID")
LoF_all.insert(0,'group',"LoF")
LoF_all.insert(1,'realgroup',"LoF")

GoF_all = pd.merge(GoF_K27, GoF_ATAC, on="ID")
GoF_all.insert(0,'group',"GoF")
GoF_all.insert(1,'realgroup',"GoF")

other_2_all = pd.merge(other_K27, other_ATAC, on="ID")
other_2_all.insert(0,'group',"other")
other_2_all.insert(1,'realgroup',"other")

LoK27_2_all = pd.merge(Variable_K27_K27, Variable_K27_ATAC, on="ID")
LoK27_2_all.insert(0,'group',"other")
LoK27_2_all.insert(1,'realgroup',"LoK27")

for_keras4 = pd.concat([other_2_all.iloc[0:20,], LoF_all, GoF_all, LoK27_2_all.iloc[0:20,]])
for_keras4.to_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/forkeras_3groups_K27_ATAC_otherLoK27.csv', sep = "\t", index=False)
print(for_keras3.shape)
for_keras4

Now, selecting windows on the interesting peaks to call variants

In [ ]:
Variable_K27 = pd.read_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/Variable_K27.tsv', sep = "\t")

Variable_K27["start-2kb"]=round((Variable_K27['Start'] + (Variable_K27['length-ATAC-union']/2) - 2000)).astype(int)
Variable_K27["end-2kb"]=round((Variable_K27['End'] + (Variable_K27['length-ATAC-union']/2) + 2000)).astype(int)

Variable_K27["start-2kb"] = Variable_K27[["start-2kb"]].astype(str)
Variable_K27["end-2kb"] = Variable_K27[["end-2kb"]].astype(str)
Variable_K27["ID"]=Variable_K27['Chr'].str.cat(Variable_K27['start-2kb'].str.cat(Variable_K27['end-2kb'],sep="-"), sep = ":")


In [ ]:
LoF = pd.read_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/Loss_of_function.tsv', sep = "\t")

LoF["start-2kb"]=round((LoF['Start'] + (LoF['length-ATAC-union']/2) - 2000)).astype(int)
LoF["end-2kb"]=round((LoF['End'] + (LoF['length-ATAC-union']/2) + 2000)).astype(int)

LoF["start-2kb"] = LoF[["start-2kb"]].astype(str)
LoF["end-2kb"] = LoF[["end-2kb"]].astype(str)
LoF["ID"]=LoF['Chr'].str.cat(LoF['start-2kb'].str.cat(LoF['end-2kb'],sep="-"), sep = ":")


In [ ]:
GoF = pd.read_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/Gain_of_function.bed', sep = "\t")

GoF["start-2kb"]=round((GoF['Start'] + (GoF['length-ATAC-union']/2) - 2000)).astype(int)
GoF["end-2kb"]=round((GoF['End'] + (GoF['length-ATAC-union']/2) + 2000)).astype(int)

GoF["start-2kb"] = GoF[["start-2kb"]].astype(str)
GoF["end-2kb"] = GoF[["end-2kb"]].astype(str)
GoF["ID"]=GoF['Chr'].str.cat(GoF['start-2kb'].str.cat(GoF['end-2kb'],sep="-"), sep = ":")

In [ ]:
Variable_K27['ID'].to_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/Variable_K27_2kb_for_snps.csv', sep = "\t", index=False, header = False)
GoF['ID'].to_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/GoF_2kb_for_snps.csv', sep = "\t", index=False, header = False)
LoF['ID'].to_csv('/t1-data/project/fgenomics/jinchaus/rotation/50Donors/peakcall/H3K27ac/sumunion/interesting_peaks/LoF_2kb_for_snps.csv', sep = "\t", index=False, header = False)
